In [ ]:
# Install necessary libraries
! pip install notebook jupyter
! pip install pandas
! pip install scikit-learn
! pip install gc
! pip install hashlib


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement gc (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for gc
ERROR: Ignored the following yanked versions: 20081119
ERROR: Could not find a version that satisfies the requirement hashlib (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for hashl

In [1]:
! pip install matplotlib


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Import libraries

In [2]:
# Import pandas for data manipulation
import pandas as pd
# Import numpy for numerical operations
import numpy as np
# Import resample for data resampling techniques
from sklearn.utils import resample
# Import preprocessing tools from scikit-learn
from sklearn import preprocessing
# Import garbage collector for memory management
import gc
# Import hashlib for cryptographic hash functions
import hashlib
# Import time module for measuring execution time
import time

# Libraries for deep learning models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Traditional machine learning libraries
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# For plotting charts (if needed)
import matplotlib.pyplot as plt

Writing functions that handle file loading and downsampling

In [ ]:
mult = 5

def load_file(path):
    # Load data from CSV file
    data = pd.read_csv(path, sep=',', low_memory=False)
    data.columns = data.columns.str.strip()  # Remove whitespace from column names
    
    # Separate normal and attack flows
    is_benign = data['Label'] == 'BENIGN'
    flows_ok = data[is_benign]
    flows_ddos_full = data[~is_benign]
    
    sizeDownSample = len(flows_ok) * mult  # Size of the final set of anomalous data
    
    # Downsample majority class
    if len(flows_ok) * mult < len(flows_ddos_full):
        flows_ddos_reduced = resample(flows_ddos_full,
                                      replace=False,  # Sample without replacement
                                      n_samples=sizeDownSample,  # Match minority n
                                      random_state=27)  # Reproducible results
    else:
        flows_ddos_reduced = flows_ddos_full
    
    return flows_ok, flows_ddos_reduced

def load_huge_file(path):
    # Add low_memory=False to avoid DtypeWarning
    df_chunk = pd.read_csv(path, chunksize=500000, low_memory=False)
    
    chunk_list_ok = []  # Append each chunk df here
    chunk_list_ddos = []
    
    # Each chunk is in df format
    for chunk in df_chunk:
        chunk.columns = chunk.columns.str.strip()  # Remove whitespace from column names
        
        is_benign = chunk['Label'] == 'BENIGN'
        flows_ok = chunk[is_benign]
        flows_ddos_full = chunk[~is_benign]
        
        sizeDownSample = len(flows_ok) * mult  # Size of the final set of anomalous data
        
        if len(flows_ok) * mult < len(flows_ddos_full):
            # Downsample majority class
            flows_ddos_reduced = resample(flows_ddos_full,
                                          replace=False,  # Sample without replacement
                                          n_samples=sizeDownSample,  # Match minority n
                                          random_state=27)  # Reproducible results
        else:
            flows_ddos_reduced = flows_ddos_full
        
        # Once the data filtering is done, append the chunk to list
        chunk_list_ok.append(flows_ok)
        chunk_list_ddos.append(flows_ddos_reduced)
        
        # Clean memory after each chunk
        del chunk, flows_ok, flows_ddos_full, flows_ddos_reduced
        gc.collect()
    
    # Concatenate the list into dataframe
    flows_ok = pd.concat(chunk_list_ok, ignore_index=True)
    flows_ddos = pd.concat(chunk_list_ddos, ignore_index=True)
    
    # Clean memory after concatenation
    del chunk_list_ok, chunk_list_ddos
    gc.collect()
    
    return flows_ok, flows_ddos

Loading CIC-DDoS2019 - Day 1 (training)

In [ ]:
# File 1
flows_ok, flows_ddos = load_huge_file(r"D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-01-12\01-12\DrDoS_DNS.csv")
print('File 1 loaded')

File 1 loaded


In [ ]:
# File 2
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-01-12\01-12\DrDoS_LDAP.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 2 loaded')

File 2 loaded


In [ ]:
# File 3
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-01-12\01-12\DrDoS_MSSQL.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 3 loaded')

File 3 loaded


In [ ]:
# File 4
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-01-12\01-12\DrDoS_NetBIOS.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 4 loaded')

File 4 loaded


In [ ]:
# File 5
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-01-12\01-12\DrDoS_NTP.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 5 loaded')

File 5 loaded


In [ ]:
# File 6
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-01-12\01-12\DrDoS_SNMP.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 6 loaded')

In [ ]:
# File 7
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-01-12\01-12\DrDoS_SSDP.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 7 loaded')

File 7 loaded


In [ ]:
# File 8
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-01-12\01-12\DrDoS_UDP.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 8 loaded')

File 8 loaded


In [ ]:
# File 9
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-01-12\01-12\Syn.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 9 loaded')

File 9 loaded


In [ ]:
# File 10
a, b = load_huge_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-01-12\01-12\TFTP.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 10 loaded')

File 10 loaded


In [ ]:
# File 11
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-01-12\01-12\UDPLag.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 11 loaded')

File 11 loaded


In [ ]:
# Combine and save results
samples = pd.concat([flows_ok, flows_ddos], ignore_index=True)
samples.to_csv(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\export_dataframe.csv', index=None, header=True)
# Clean up memory
del flows_ddos, flows_ok, a, b

Loading CIC-DDoS2019 - Day 2 (testing)

In [ ]:
# file 1
flows_ok, flows_ddos = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-03-11\03-11\LDAP.csv')
print('file 1 loaded')

file 1 loaded


In [ ]:
# File 2
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-03-11\03-11\MSSQL.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 2 loaded')

File 2 loaded


In [ ]:
# File 3
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-03-11\03-11\NetBIOS.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 3 loaded')

File 3 loaded


In [ ]:
# File 4
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-03-11\03-11\Portmap.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 4 loaded')

File 4 loaded


In [ ]:
# File 5
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-03-11\03-11\Syn.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 5 loaded')

File 5 loaded


In [ ]:
# File 6
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-03-11\03-11\UDP.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 6 loaded')

File 6 loaded


In [ ]:
# File 7
a, b = load_file(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\CSV-03-11\03-11\UDPLag.csv')
flows_ok = pd.concat([flows_ok, a], ignore_index=True)
flows_ddos = pd.concat([flows_ddos, b], ignore_index=True)
print('File 7 loaded')

File 7 loaded


In [ ]:
tests = pd.concat([flows_ok, flows_ddos], ignore_index=True)
tests.to_csv(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\export_tests.csv.csv', index=None, header=True)
del flows_ddos, flows_ok, a, b

CIC-DDoS2019 Data Processing

In [ ]:
# Read data from CSV file
samples = pd.read_csv(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\export_dataframe.csv', sep=',', low_memory=False)

# Function to encode string to numeric using MD5
def string2numeric_hash(text):
   return int(hashlib.md5(text).hexdigest()[:8], 16)

# Replace invalid values
samples = samples.replace('Infinity', '0')
samples = samples.replace(np.inf, 0)

# Process 'Flow Packets/s' column
samples['Flow Packets/s'] = pd.to_numeric(samples['Flow Packets/s'])

# Process 'Flow Bytes/s' column
samples['Flow Bytes/s'] = samples['Flow Bytes/s'].fillna(0)
samples['Flow Bytes/s'] = pd.to_numeric(samples['Flow Bytes/s'])

# Encode labels: BENIGN = 0, attacks = 1
attack_labels = [
   'DrDoS_DNS', 'DrDoS_LDAP', 'DrDoS_MSSQL', 'DrDoS_NTP',
   'DrDoS_NetBIOS', 'DrDoS_SNMP', 'DrDoS_SSDP', 'DrDoS_UDP',
   'Syn', 'TFTP', 'UDP-lag', 'WebDDoS'
]
samples['Label'] = samples['Label'].replace('BENIGN', 0)
samples['Label'] = samples['Label'].replace(attack_labels, 1)
samples['Label'] = samples['Label'].astype(int)

# Process 'Timestamp' column
samples['Timestamp'] = samples['Timestamp'].astype(str)

# Split date and time from Timestamp
time_parts = samples['Timestamp'].str.split(' ', n=1, expand=True)
time_parts.columns = ['date', 'time']

# Split hour and millisecond (if present)
time_split = time_parts['time'].str.split('.', n=1, expand=True)
time_split.columns = ['hour', 'millisecond']

# Encode hour using MD5 and convert to number
hour_encoded = time_split['hour'].str.encode('utf-8')
samples['Timestamp'] = hour_encoded.apply(lambda x: string2numeric_hash(x))

# Delete temporary variables
del time_parts, time_split, hour_encoded

C:\Users\HP\AppData\Local\Temp\ipykernel_19920\611314941.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  samples['Label'] = samples['Label'].replace(attack_labels, 1)


In [ ]:
del samples['Source IP']
del samples['Destination IP']
del samples['Flow ID']
del samples['SimillarHTTP']
del samples['Unnamed: 0']

In [ ]:
samples.to_csv(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\export_dataframe_proc.csv', index = None, header=True)
print('Training data processed')

Training data processed


In [ ]:
# Define function to hash string to numeric value (MD5)
def string2numeric_hash(text):
   return int(hashlib.md5(text).hexdigest()[:8], 16)

# Read test data
tests = pd.read_csv(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\export_tests.csv.csv', sep=',', low_memory=False)

# Handle Infinity and NaN values
tests = tests.replace('Infinity', '0')
tests = tests.replace(np.inf, 0)

# Convert 'Flow Packets/s' and 'Flow Bytes/s' columns to numeric type
tests['Flow Packets/s'] = pd.to_numeric(tests['Flow Packets/s'], errors='coerce').fillna(0)
tests['Flow Bytes/s'] = pd.to_numeric(tests['Flow Bytes/s'], errors='coerce').fillna(0)

# Assign labels: BENIGN = 0, attack types = 1 (same as training file)
attack_labels = [
   'DrDoS_DNS', 'DrDoS_LDAP', 'DrDoS_MSSQL', 'DrDoS_NTP',
   'DrDoS_NetBIOS', 'DrDoS_SNMP', 'DrDoS_SSDP', 'DrDoS_UDP',
   'Syn', 'TFTP', 'UDP-lag', 'WebDDoS', 'LDAP', 'NetBIOS', 'MSSQL', 'Portmap', 'UDP', 'UDPLag'
]

# Convert 'Label' column to string type
tests['Label'] = tests['Label'].astype(str)

# Replace 'BENIGN' with 0 and attack labels with 1
tests['Label'] = tests['Label'].replace('BENIGN', 0)
tests['Label'] = tests['Label'].apply(lambda x: 1 if x in attack_labels else 0)

# Convert 'Label' column to int type after replacement
tests['Label'] = tests['Label'].astype(int)

# Process 'Timestamp' column -> Encode hour
tests['Timestamp'] = tests['Timestamp'].astype(str)
time_parts = tests['Timestamp'].str.split(' ', n=1, expand=True)
time_parts.columns = ['date', 'time']
time_split = time_parts['time'].str.split('.', n=1, expand=True)
time_split.columns = ['hour', 'millisecond']

# Encode hour to MD5 hash and convert to number
hour_encoded = time_split['hour'].str.encode('utf-8')
tests['Timestamp'] = hour_encoded.apply(lambda x: string2numeric_hash(x))

# Delete unnecessary temporary variables
del time_parts, time_split, hour_encoded

In [ ]:
# Remove unnecessary columns
cols_to_drop = ['Source IP', 'Destination IP', 'Flow ID', 'SimillarHTTP', 'Unnamed: 0']
tests.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# Save the processed file
tests.to_csv(r'D:\Nam3\HK2\ATBM\DDOS\Dataset\export_tests_proc.csv', index=False, header=True)
print('Test data processed')

Test data processed
